# Fastai 2019: Lesson 8 - Dev Setup

Lessons can be found : https://github.com/fastai/fastai_docs/tree/master/dev_course/dl2

Covering:
- https://github.com/fastai/fastai_docs/blob/master/dev_course/dl2/00_exports.ipynb

These notes will try and follow the lesson as close as possible, without jumping around to too many files. For this particular course, the libraries being built will be done via Notebooks, instead of the traditional `.py` files. There's currently a helper file in this directory called `notebook2script.py` that will convert cells into functions

### Sample export cell

Prerequistes: `pip install fire`

```
mkdir exp
```

What to run:

```
$python notebook2script.py 00_exports.ipynb
```

This will go through a particular notebook, and will find the cells with `#exports`

In [ ]:
#export
TEST = 'test'

In [1]:
!mkdir exp

mkdir: exp: File exists


In [11]:
!python notebook2script.py lesson81.ipynb

Converted lesson81.ipynb to nb_lesson81.py


### Load the saved constant

Now that the above should have been created, we should be able to import the variable `TEST`. For the purpose of notes, the functions will be written in a longer format instead of the compact version found in the lesson

In [3]:
from exp.nb_lesson8 import *
import operator


def test(a, b, cmp, cname=None):
    """
    Simple test function
    """
    # if no name is passed, 
    if cname is None: 
        cname=cmp.__name__

    # this is the test
    # if it fails, the second phrase will be returned
    # which is the f"{cname}:\n{a}\n{b}"
    assert cmp(a, b), f"{cname}:\n{a}\n{b}"

def test_eq(a,b):
    """
    Is a larger function that calls the test function
    but only passes teh == sign
    """
    test(a, b, operator.eq, '==')

### Run a sample test

If correct, you won't get any feedback

```python
test_eq(TEST, "test")
```

If it was wrong you should get a message as follows:
    
```python
test_eq(TEST, "nottest")
```

```bash
    -
    AssertionErrorTraceback (most recent call last)
    <ipython-input-5-24a85abd2d33> in <module>
    ----> 1 test_eq(TEST, "nottest")

    <ipython-input-3-82ec92dcee6d> in test_eq(a, b)
         21     but only passes teh == sign
         22     """
    ---> 23     test(a, b, operator.eq, '==')

    <ipython-input-3-82ec92dcee6d> in test(a, b, cmp, cname)
         14     # if it fails, the second phrase will be returned
         15     # which is the f"{cname}:\n{a}\n{b}"
    ---> 16     assert cmp(a, b), f"{cname}:\n{a}\n{b}"
         17 
         18 def test_eq(a,b):

    AssertionError: ==:
    test
    nottest

```
    

In [4]:
test_eq(TEST, "test")

### How to run tests outside the notebook

`run_notebook.py` will do this

```python
python run_notebook.py lesson8.ipynb
```

### What's in `run_notebook.py`


```python
#!/usr/bin/env python

import nbformat,fire
from nbconvert.preprocessors import ExecutePreprocessor

def run_notebook(path):
    nb = nbformat.read(open(path), as_version=nbformat.NO_CONVERT)
    ExecutePreprocessor(timeout=600).preprocess(nb, {})
    print('done')

if __name__ == '__main__': fire.Fire(run_notebook)
```

- **`nbformat`** lets you run a notebook and print out any errors
- **`fire`** takes any function and turns it into a command line interface, this manages arguments and other input values

#### Example

```python
! python run_notebook.py
```

Will return the function inputs / parameters

```bash
    Fire trace:
    1. Initial component
    2. ('The function received no value for the required argument:', 'path')

    Type:        function
    String form: <function run_notebook at 0x102a03f28>
    File:        ~/myrepos/fastai_dl2019p2/live_notes/run_notebook.py
    Line:        6

    Usage:       run_notebook.py PATH
                 run_notebook.py --path PATH
```

### What's in `notebook2script.py`

```python
#!/usr/bin/env python

import json,fire,re
from pathlib import Path

def is_export(cell):
    if cell['cell_type'] != 'code': return False
    src = cell['source']
    if len(src) == 0 or len(src[0]) < 7: return False
    #import pdb; pdb.set_trace()
    return re.match(r'^\s*#\s*export\s*$', src[0], re.IGNORECASE) is not None

def notebook2script(fname):
    fname = Path(fname)
    fname_out = f'nb_{fname.stem.split("_")[0]}.py'
    main_dic = json.load(open(fname,'r'))
    code_cells = [c for c in main_dic['cells'] if is_export(c)]
    module = f'''
#################################################
### THIS FILE WAS AUTOGENERATED! DO NOT EDIT! ###
#################################################
# file to edit: dev_nb/{fname.name}
'''
    for cell in code_cells: module += ''.join(cell['source'][1:]) + '\n\n'
    # remove trailing spaces
    module = re.sub(r' +$', '', module, flags=re.MULTILINE)
    open(fname.parent/'exp'/fname_out,'w').write(module[:-2])
    print(f"Converted {fname} to {fname_out}")

if __name__ == '__main__': fire.Fire(notebook2script)
```

One of the key observations is that notebooks are json. Hence the line:

```python
main_dic = json.load(open(fname,'r'))
```

For example, loading this lesson notebook can be done by the following:

```python
import json
notebook_as_json_cells = json.load(open("lesson8.ipynb","r"))["cells"]
notebook_as_json_cells[2]
```

```bash
{'cell_type': 'code',
 'execution_count': None,
 'metadata': {},
 'outputs': [],
 'source': ['#export\n', "TEST = 'test'"]}
```